In [128]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from utils import read_config, MySQLAgent
import time, random

In [129]:
config = read_config('.env/local_conn.json')
sql_agent = MySQLAgent(config['local_mysql'])

In [130]:
query = """
show tables
"""

sql_agent.read_table(query=query)

,tables_in_generaldb
0,balancesheet
1,port_activity
2,profitandlose


### Helper functions

In [131]:
def get_acct_data_and_rename(df, row_name, new_name):

    result = df[df['acct_name'] == row_name][['year_season', 'this_year_amt']].reset_index(drop=True).rename(columns={'this_year_amt':new_name})
    result[new_name] = result[new_name].str.replace(',', '').astype(int)

    return result

In [132]:
query = """
select *
from balancesheet
where stock = '2330' # and season = 4
"""

df_balance = sql_agent.read_table(query=query)

In [133]:
df_balance

,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,report_name,report_time_raw,period_year,season,creation_date,year_season,stock
0,流動資產,,,,,BalanceSheet,民國108年第1季,108,1,2024-09-07,108-1,2330
1,現金及約當現金,"645,670,527",29.52,"577,782,963",28.20,BalanceSheet,民國108年第1季,108,1,2024-09-07,108-1,2330
2,透過損益按公允價值衡量之金融資產－流動,"3,084,399",0.14,"963,915",0.05,BalanceSheet,民國108年第1季,108,1,2024-09-07,108-1,2330
3,透過其他綜合損益按公允價值衡量之金融資產－流動,"107,313,205",4.91,"95,713,446",4.67,BalanceSheet,民國108年第1季,108,1,2024-09-07,108-1,2330
4,按攤銷後成本衡量之金融資產－流動,"4,179,386",0.19,"9,888,741",0.48,BalanceSheet,民國108年第1季,108,1,2024-09-07,108-1,2330
...,...,...,...,...,...,...,...,...,...,...,...,...
1393,權益總額,"3,665,716,254",63.33,"3,092,897,598",61.30,BalanceSheet,民國113年第1季,113,1,2024-09-07,113-1,2330
1394,負債及權益總計,"5,787,891,082",100.00,"5,045,844,348",100.00,BalanceSheet,民國113年第1季,113,1,2024-09-07,113-1,2330
1395,待註銷股本股數（單位：股）,"1,401,750",,"419,466",,BalanceSheet,民國113年第1季,113,1,2024-09-07,113-1,2330
1396,預收股款（權益項下）之約當發行股數（單位：股）,0,,0,,BalanceSheet,民國113年第1季,113,1,2024-09-07,113-1,2330


In [134]:
df_balance.acct_name.unique()

array(['流動資產', '現金及約當現金', '透過損益按公允價值衡量之金融資產－流動',
       '透過其他綜合損益按公允價值衡量之金融資產－流動', '按攤銷後成本衡量之金融資產－流動', '避險之金融資產－流動',
       '應收帳款淨額', '應收帳款－關係人淨額', '其他應收款－關係人淨額', '存貨', '其他流動資產', '流動資產合計',
       '非流動資產', '透過其他綜合損益按公允價值衡量之金融資產－非流動', '按攤銷後成本衡量之金融資產－非流動',
       '採用權益法之投資', '不動產、廠房及設備', '使用權資產', '無形資產', '遞延所得稅資產', '其他非流動資產',
       '非流動資產合計', '資產總額', '流動負債', '短期借款', '透過損益按公允價值衡量之金融負債－流動',
       '避險之金融負債－流動', '應付帳款', '應付帳款－關係人', '其他應付款', '本期所得稅負債', '其他流動負債',
       '流動負債合計', '非流動負債', '應付公司債', '遞延所得稅負債', '租賃負債－非流動', '其他非流動負債',
       '非流動負債合計', '負債總額', '歸屬於母公司業主之權益', '股本', '普通股股本', '股本合計', '資本公積',
       '資本公積－發行溢價', '資本公積-認列對子公司所有權權益變動數', '資本公積－受贈資產',
       '資本公積－採用權益法認列關聯企業及合資股權淨值之變動數', '資本公積－合併溢額', '資本公積合計', '保留盈餘',
       '法定盈餘公積', '特別盈餘公積', '未分配盈餘（或待彌補虧損）', '保留盈餘合計', '其他權益', '其他權益合計',
       '歸屬於母公司業主之權益合計', '非控制權益', '權益總額', '負債及權益總計',
       '預收股款（權益項下）之約當發行股數（單位：股）', '母公司暨子公司所持有之母公司庫藏股股數（單位：股）', '應付短期票券',
       '長期借款', '資本公積－實際取得或處分子公司股權價格與帳面價值差額', '資本公積－限制員工權利股票', '庫藏股票',
      

In [135]:
query = """
select *
from profitandlose
where stock = '2330' # and season = 4
"""

df_profitloss = sql_agent.read_table(query=query)

In [136]:
df_profitloss.iloc[:25]

,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,report_name,report_time_raw,period_year,season,creation_date,year_season,stock
0,營業收入合計,"218,704,469",100.00,"248,078,671",100.00,ProfitAndLose,民國108年第1季,108,1,2024-09-07,108-1,2330
1,營業成本合計,"128,352,344",58.69,"123,103,977",49.62,ProfitAndLose,民國108年第1季,108,1,2024-09-07,108-1,2330
2,營業毛利（毛損）,"90,352,125",41.31,"124,974,694",50.38,ProfitAndLose,民國108年第1季,108,1,2024-09-07,108-1,2330
3,已實現銷貨（損）益,"5,432",0.00,"-117,155",-0.05,ProfitAndLose,民國108年第1季,108,1,2024-09-07,108-1,2330
4,營業毛利（毛損）淨額,"90,357,557",41.31,"124,857,539",50.33,ProfitAndLose,民國108年第1季,108,1,2024-09-07,108-1,2330
5,營業費用,,,,,ProfitAndLose,民國108年第1季,108,1,2024-09-07,108-1,2330
6,推銷費用,"1,459,973",0.67,"1,448,092",0.58,ProfitAndLose,民國108年第1季,108,1,2024-09-07,108-1,2330
7,管理費用,"4,140,729",1.89,"4,851,708",1.96,ProfitAndLose,民國108年第1季,108,1,2024-09-07,108-1,2330
8,研究發展費用,"20,417,311",9.34,"20,428,594",8.23,ProfitAndLose,民國108年第1季,108,1,2024-09-07,108-1,2330
9,營業費用合計,"26,018,013",11.90,"26,728,394",10.77,ProfitAndLose,民國108年第1季,108,1,2024-09-07,108-1,2330


In [137]:
df_profitloss[(df_profitloss['year_season'] == '112-4')].iloc[:25]

,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,report_name,report_time_raw,period_year,season,creation_date,year_season,stock
1108,營業收入合計,625528856,None,None,None,None,None,112,4,2024-09-07,112-4,2330
1109,營業成本合計,293760931,None,None,None,None,None,112,4,2024-09-07,112-4,2330
1110,營業毛利（毛損）,331767925,None,None,None,None,None,112,4,2024-09-07,112-4,2330
1111,營業毛利（毛損）淨額,331767925,None,None,None,None,None,112,4,2024-09-07,112-4,2330
1112,推銷費用,2975547,None,None,None,None,None,112,4,2024-09-07,112-4,2330
1113,管理費用,18235380,None,None,None,None,None,112,4,2024-09-07,112-4,2330
1114,研究發展費用,50410094,None,None,None,None,None,112,4,2024-09-07,112-4,2330
1115,營業費用合計,71621021,None,None,None,None,None,112,4,2024-09-07,112-4,2330
1116,其他收益及費損淨額,57599,None,None,None,None,None,112,4,2024-09-07,112-4,2330
1117,營業利益（損失）,260204503,None,None,None,None,None,112,4,2024-09-07,112-4,2330


# Financial Structure

### Debut Burden Ratio(DBR) (資產負債比)

In [138]:
# 資產總額
df_total_assets = get_acct_data_and_rename(df_balance, '資產總額', 'total_assets')
df_total_assets['total_assets_varation'] = df_total_assets['total_assets'].pct_change()*100
df_total_assets

,year_season,total_assets,total_assets_varation
0,108-1,2187436785,NaN
1,108-2,2239343671,2.372955
2,108-3,2134234450,-4.693751
3,108-4,2264805032,6.117912
4,109-1,2343295696,3.465670
5,109-2,2448664522,4.496608
6,109-3,2635572214,7.633046
7,109-4,2760711405,4.748084
8,110-1,2919131461,5.738378
9,110-2,3092815189,5.949843


In [139]:
# 負債總額
df_total_liability = get_acct_data_and_rename(df_balance, '負債總額', 'total_liabilities')
df_total_liability['total_liabilities_varation'] = df_total_liability['total_liabilities'].pct_change()*100
df_total_liability

,year_season,total_liabilities,total_liabilities_varation
0,108-1,443919183,NaN
1,108-2,684922144,54.289828
2,108-3,546737449,-20.175241
3,108-4,642709606,17.553610
4,109-1,666267165,3.665350
5,109-2,727451140,9.183099
6,109-3,847305839,16.475979
7,109-4,910089406,7.409788
8,110-1,978134499,7.476748
9,110-2,1098844616,12.340851


In [140]:
# DBR: Debut Burden Ratio
df_total_assets_liability = pd.merge(df_total_assets, df_total_liability, how='inner', on='year_season')
df_total_assets_liability['DBR'] = (round(df_total_assets_liability['total_liabilities'].astype(int) / df_total_assets_liability['total_assets'].astype(int) * 100, 2)).astype(str) + '%'
df_total_assets_liability

,year_season,total_assets,total_assets_varation,total_liabilities,total_liabilities_varation,DBR
0,108-1,2187436785,NaN,443919183,NaN,20.29%
1,108-2,2239343671,2.372955,684922144,54.289828,30.59%
2,108-3,2134234450,-4.693751,546737449,-20.175241,25.62%
3,108-4,2264805032,6.117912,642709606,17.553610,28.38%
4,109-1,2343295696,3.465670,666267165,3.665350,28.43%
5,109-2,2448664522,4.496608,727451140,9.183099,29.71%
6,109-3,2635572214,7.633046,847305839,16.475979,32.15%
7,109-4,2760711405,4.748084,910089406,7.409788,32.97%
8,110-1,2919131461,5.738378,978134499,7.476748,33.51%
9,110-2,3092815189,5.949843,1098844616,12.340851,35.53%


- 108年~110年，債務比例增加，可能是要擴廠或增加設備等等

### Long term funds to fixed assets ratio(長期資金佔固定資產比率)

(股東權益淨額 + 長期負債) / 固定資產淨額

在評估企業財務穩健性時，長期資金佔固定資產比率是一個重要的指標。長期資金，如股本和長期負債，被視為相對穩定的資金來源，而固定資產是指長期持有、無法迅速變現的資產，如廠房和設備。長期資金佔固定資產比率衡量長期資金對固定資產的覆蓋程度，反映企業償還固定資產投資的能力。

一般而言，較高的長期資金佔固定資產比率表示企業的財務結構較為穩健，因為它意味著企業有更多資金可供償還固定資產投資。然而，過高的比率也並非好事。當長期資金過多時，企業可能會無法有效利用資金，導致資金利用率低下，進而影響盈利能力。因此，在評估企業的財務穩健性時，投資者需要綜合考量長期資金佔固定資產比率和其他財務指標，例如負債對股東權益比率和現金流量狀況，才能做出全面判斷。

In [141]:
# 固定資產: 不動產、廠房及設備
df_fixed_assets = get_acct_data_and_rename(df_balance, '不動產、廠房及設備', 'fixed_assets')
df_fixed_assets

,year_season,fixed_assets
0,108-1,1107651816
1,108-2,1142871184
2,108-3,1197955298
3,108-4,1352377405
4,109-1,1438215285
5,109-2,1493560950
6,109-3,1504438119
7,109-4,1555589120
8,110-1,1658551290
9,110-2,1722679133


In [142]:
# 長期借款
df_long_term_liabilities = get_acct_data_and_rename(df_balance, '長期借款', 'long-term_liabilities')
df_long_term_liabilities

,year_season,long-term_liabilities
0,109-3,982449
1,109-4,1967611
2,110-1,2623824
3,110-2,3171331
4,110-3,3472190
5,110-4,3309131
6,111-1,3087714
7,111-2,3318773
8,111-3,3055178
9,111-4,4760047


In [143]:
df_balance[df_balance['acct_name'] == '長期借款']

,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,report_name,report_time_raw,period_year,season,creation_date,year_season,stock
421,長期借款,"982,449",0.04,0,0.00,BalanceSheet,民國109年第3季,109,3,2024-09-07,109-3,2330
486,長期借款,"1,967,611",0.07,0,0.00,BalanceSheet,民國109年第4季,109,4,2024-09-07,109-4,2330
552,長期借款,"2,623,824",0.09,0,0.00,BalanceSheet,民國110年第1季,110,1,2024-09-07,110-1,2330
619,長期借款,"3,171,331",0.10,0,0.00,BalanceSheet,民國110年第2季,110,2,2024-09-07,110-2,2330
685,長期借款,"3,472,190",0.10,"982,449",0.04,BalanceSheet,民國110年第3季,110,3,2024-09-07,110-3,2330
751,長期借款,"3,309,131",0.09,"1,967,611",0.07,BalanceSheet,民國110年第4季,110,4,2024-09-07,110-4,2330
817,長期借款,"3,087,714",0.08,"2,623,824",0.09,BalanceSheet,民國111年第1季,111,1,2024-09-07,111-1,2330
885,長期借款,"3,318,773",0.08,"3,171,331",0.10,BalanceSheet,民國111年第2季,111,2,2024-09-07,111-2,2330
953,長期借款,"3,055,178",0.07,"3,472,190",0.10,BalanceSheet,民國111年第3季,111,3,2024-09-07,111-3,2330
1021,長期借款,"4,760,047",0.10,"3,309,131",0.09,BalanceSheet,民國111年第4季,111,4,2024-09-07,111-4,2330


In [144]:
# 股東權益＝資產－負債
df_shareholder_equity = df_total_assets_liability[['year_season', 'total_assets', 'total_liabilities']].copy()
df_shareholder_equity['shareholder_equity'] = df_shareholder_equity['total_assets'] - df_shareholder_equity['total_liabilities']
df_shareholder_equity

,year_season,total_assets,total_liabilities,shareholder_equity
0,108-1,2187436785,443919183,1743517602
1,108-2,2239343671,684922144,1554421527
2,108-3,2134234450,546737449,1587497001
3,108-4,2264805032,642709606,1622095426
4,109-1,2343295696,666267165,1677028531
5,109-2,2448664522,727451140,1721213382
6,109-3,2635572214,847305839,1788266375
7,109-4,2760711405,910089406,1850621999
8,110-1,2919131461,978134499,1940996962
9,110-2,3092815189,1098844616,1993970573


In [145]:
parent_table = df_balance[['year_season']].drop_duplicates()
df_temp1 = pd.merge(df_fixed_assets, df_long_term_liabilities, how='left', on='year_season')
df_temp2 = pd.merge(df_temp1, df_shareholder_equity, how='left', on='year_season')
# in here, we should check the last amt in the original table. 0 here is the result after checking
# create a check function laterly
df_longterm_funds_to_fixed_assets_retio = pd.merge(parent_table, df_temp2, how='left', on='year_season').fillna(0)
df_longterm_funds_to_fixed_assets_retio['longterm_funds_to_fixed_assets_retio'] = round((df_longterm_funds_to_fixed_assets_retio['long-term_liabilities'].astype(int) + df_longterm_funds_to_fixed_assets_retio['shareholder_equity'])/ df_longterm_funds_to_fixed_assets_retio['fixed_assets'].astype(int), 2)
df_longterm_funds_to_fixed_assets_retio

,year_season,fixed_assets,long-term_liabilities,total_assets,total_liabilities,shareholder_equity,longterm_funds_to_fixed_assets_retio
0,108-1,1107651816,0.0,2187436785,443919183,1743517602,1.57
1,108-2,1142871184,0.0,2239343671,684922144,1554421527,1.36
2,108-3,1197955298,0.0,2134234450,546737449,1587497001,1.33
3,108-4,1352377405,0.0,2264805032,642709606,1622095426,1.20
4,109-1,1438215285,0.0,2343295696,666267165,1677028531,1.17
5,109-2,1493560950,0.0,2448664522,727451140,1721213382,1.15
6,109-3,1504438119,982449.0,2635572214,847305839,1788266375,1.19
7,109-4,1555589120,1967611.0,2760711405,910089406,1850621999,1.19
8,110-1,1658551290,2623824.0,2919131461,978134499,1940996962,1.17
9,110-2,1722679133,3171331.0,3092815189,1098844616,1993970573,1.16


- The longterm_funds_to_fixed_assets_ratio for TSMC has fluctuated slightly between 1.10 and 1.20 over the past five years, indicating a balanced and consistent financial strategy.
- The slight decrease in the ratio in 2021 and 2022 reflects strategic investments in fixed assets, possibly for expansion or upgrading capabilities.
- The increase in the ratio in 2023 suggests a return to a slightly more conservative approach, increasing long-term funds relative to fixed assets, which enhances financial stability.
- The consistent ratios indicate that TSMC effectively manages its long-term funds to ensure sufficient coverage for its fixed assets, supporting sustained growth and operational resilience.

# Solvency (償債能力)

### Current ratio (流動比率)

Current ratio = current assets / current liabilities

流動比率 = 流動資產 / 流動負債

In [146]:
df_current_assets = get_acct_data_and_rename(df_balance, '流動資產合計', 'current_assets')
df_current_assets

,year_season,current_assets
0,108-1,991324815
1,108-2,1010179338
2,108-3,849427436
3,108-4,822613914
4,109-1,802470040
5,109-2,860031580
6,109-3,1029673918
7,109-4,1092185308
8,110-1,1139786823
9,110-2,1241104023


In [147]:
df_current_liabilities = get_acct_data_and_rename(df_balance, '流動負債合計', 'current_liabilities')
df_current_liabilities

,year_season,current_liabilities
0,108-1,378267634
1,108-2,622256378
2,108-3,494781125
3,108-4,590735701
4,109-1,589468059
5,109-2,614795728
6,109-3,588241723
7,109-4,617151048
8,110-1,661648612
9,110-2,647748947


In [148]:
df_assets_liabilities = pd.merge(df_current_assets, df_current_liabilities, how='inner', on='year_season')
df_assets_liabilities['current_ratio'] = (round(df_assets_liabilities['current_assets'] / df_assets_liabilities['current_liabilities'] *100, 2)).astype(str) + '%'
df_assets_liabilities

,year_season,current_assets,current_liabilities,current_ratio
0,108-1,991324815,378267634,262.07%
1,108-2,1010179338,622256378,162.34%
2,108-3,849427436,494781125,171.68%
3,108-4,822613914,590735701,139.25%
4,109-1,802470040,589468059,136.13%
5,109-2,860031580,614795728,139.89%
6,109-3,1029673918,588241723,175.04%
7,109-4,1092185308,617151048,176.97%
8,110-1,1139786823,661648612,172.26%
9,110-2,1241104023,647748947,191.6%


- Current ratio is bigger than 100% which is healthy.
- The meaning of healthy is that the company has the ability to pay the liabilities.

流動比率大於100%為佳， 若小於100%須瞭解流動資產的變現性， 還有短期金融負債(短期借款、應附商業本票和一年內到期長期負債)佔流動負債的比例。 如果流動資產變現性很高，短期金融負債佔流動負債的比例低， 則低流動比率可接受；反之則償債能力不佳。 短期償債壓力越大，投資應特別注意。

In [149]:
# def for calculating short term liabilities

### Quick Ratio(速動比率)

速動比率 =（流動資產-存貨）/流動負債

In [150]:
# 存貨
df_inventories = get_acct_data_and_rename(df_balance, '存貨', 'inventories')
df_inventories

,year_season,inventories
0,108-1,108682382
1,108-2,108231879
2,108-3,96685730
3,108-4,82981196
4,109-1,78277834
5,109-2,85788094
6,109-3,109509486
7,109-4,137353407
8,110-1,154567568
9,110-2,170438686


In [151]:
df_assets_liabilities = pd.merge(df_current_assets, df_current_liabilities, how='inner', on='year_season')
df_assets_liabilities_inventories = pd.merge(df_assets_liabilities, df_inventories, how='inner', on='year_season')
df_assets_liabilities_inventories['quick_ratio'] = round((df_assets_liabilities_inventories['current_assets'] - df_assets_liabilities_inventories['inventories'])/df_assets_liabilities_inventories['current_liabilities']*100, 2).astype(str) + '%'
df_assets_liabilities_inventories

,year_season,current_assets,current_liabilities,inventories,quick_ratio
0,108-1,991324815,378267634,108682382,233.34%
1,108-2,1010179338,622256378,108231879,144.95%
2,108-3,849427436,494781125,96685730,152.14%
3,108-4,822613914,590735701,82981196,125.21%
4,109-1,802470040,589468059,78277834,122.86%
5,109-2,860031580,614795728,85788094,125.94%
6,109-3,1029673918,588241723,109509486,156.43%
7,109-4,1092185308,617151048,137353407,154.72%
8,110-1,1139786823,661648612,154567568,148.9%
9,110-2,1241104023,647748947,170438686,165.29%


# Operating ability(經營能力)

### Receivables Turnover Ratio(應收款項週轉率)

- 銷貨淨額: Revenue 
- 各期平均應收款項餘額: Average Accounts Receivable
- DSO = (Average Accounts Receivable) / (Revenue) * 365

- Receivables Turnover Ratio = (Revenue) / Average Accounts Receivable
- 應收款項週轉率 = 銷貨淨額 / 各期平均應收款項餘額

In [152]:
# 應收帳款淨額
df_receivables = get_acct_data_and_rename(df_balance, '應收帳款淨額', 'receivables')
df_receivables

,year_season,receivables
0,108-1,106431149
1,108-2,115725802
2,108-3,144240857
3,108-4,138908589
4,109-1,145992744
5,109-2,148708055
6,109-3,161700332
7,109-4,145480272
8,110-1,167143336
9,110-2,177475493


In [153]:
# 銷貨淨額
df_net_revenue = get_acct_data_and_rename(df_profitloss, '營業毛利（毛損）淨額', 'net_revenue')
df_net_revenue

,year_season,net_revenue
0,108-1,90357557
1,108-2,103730060
2,108-3,139412434
3,108-year,492701896
4,109-1,160776508
5,109-2,164623791
6,109-3,190493496
7,109-year,711130120
8,110-1,189839430
9,110-2,186196818


In [154]:
df_receivable_turnover = pd.merge(df_receivables, df_net_revenue, how='inner', on='year_season')
df_receivable_turnover['receivable_turnover_ratio'] = (df_receivable_turnover['receivables'] / df_receivable_turnover['net_revenue'])
df_receivable_turnover

,year_season,receivables,net_revenue,receivable_turnover_ratio
0,108-1,106431149,90357557,1.177889
1,108-2,115725802,103730060,1.115644
2,108-3,144240857,139412434,1.034634
3,108-4,138908589,159201845,0.872531
4,109-1,145992744,160776508,0.908048
5,109-2,148708055,164623791,0.903321
6,109-3,161700332,190493496,0.848850
7,109-4,145480272,195236325,0.745150
8,110-1,167143336,189839430,0.880446
9,110-2,177475493,186196818,0.953161


### Inventory Turnover(存貨周轉率)

公司在某一段時間的營業成本與平均存貨餘額的比例，可以反應存貨的周轉速度，藉此看出存貨流動性、存貨資金占用量是否合理。

存貨周轉率 = 營業成本 / 平均存貨 = 營業成本 / ((期初存貨 + 期末存貨) / 2)

Inventroy Turnover = 

In [155]:
df_operation_cost = get_acct_data_and_rename(df_profitloss, '營業成本合計', 'operation_cost')
df_operation_cost

,year_season,operation_cost
0,108-1,128352344
1,108-2,137325245
2,108-3,153613278
3,108-year,577286947
4,109-1,149813002
5,109-2,146005542
6,109-3,165945534
7,109-year,628108309
8,110-1,172635011
9,110-2,185910987


In [156]:
df_inventories = get_acct_data_and_rename(df_balance, '存貨', 'inventories')
df_inventories

,year_season,inventories
0,108-1,108682382
1,108-2,108231879
2,108-3,96685730
3,108-4,82981196
4,109-1,78277834
5,109-2,85788094
6,109-3,109509486
7,109-4,137353407
8,110-1,154567568
9,110-2,170438686


In [157]:
df_inventories['avg_inventories'] = (df_inventories['inventories'] + df_inventories['inventories'].shift(1)) / 2
df_inventories['avg_inventories'] = np.where(df_inventories['avg_inventories'].isnull(), df_inventories['inventories'], df_inventories['avg_inventories'])
df_inventories

,year_season,inventories,avg_inventories
0,108-1,108682382,108682382.0
1,108-2,108231879,108457130.5
2,108-3,96685730,102458804.5
3,108-4,82981196,89833463.0
4,109-1,78277834,80629515.0
5,109-2,85788094,82032964.0
6,109-3,109509486,97648790.0
7,109-4,137353407,123431446.5
8,110-1,154567568,145960487.5
9,110-2,170438686,162503127.0


In [158]:
df_inventories_turnover = pd.merge(df_inventories, df_operation_cost, how='inner', on='year_season')
df_inventories_turnover['inventories_turnover'] = round(df_inventories_turnover['operation_cost'] / df_inventories_turnover['avg_inventories'], 2)
# Days sales of invetory
df_inventories_turnover['DSI'] = round(365 / df_inventories_turnover['inventories_turnover'], 2)
df_inventories_turnover

,year_season,inventories,avg_inventories,operation_cost,inventories_turnover,DSI
0,108-1,108682382,108682382.0,128352344,1.18,309.32
1,108-2,108231879,108457130.5,137325245,1.27,287.40
2,108-3,96685730,102458804.5,153613278,1.50,243.33
3,108-4,82981196,89833463.0,157996080,1.76,207.39
4,109-1,78277834,80629515.0,149813002,1.86,196.24
5,109-2,85788094,82032964.0,146005542,1.78,205.06
6,109-3,109509486,97648790.0,165945534,1.70,214.71
7,109-4,137353407,123431446.5,166344231,1.35,270.37
8,110-1,154567568,145960487.5,172635011,1.18,309.32
9,110-2,170438686,162503127.0,185910987,1.14,320.18


### Asset Turnover Ratio(資產周轉率)


- 資產周轉率 = 總營業收入(Total Revenue) ÷ 總資產(Total Asset)
- 總資產(Total Asset) = (期初資產+期末資產) ÷2



In [159]:
df_total_revenue = get_acct_data_and_rename(df_profitloss, '營業收入合計', 'total_revenue')
df_total_revenue

,year_season,total_revenue
0,108-1,218704469
1,108-2,240998475
2,108-3,293045439
3,108-year,1069985448
4,109-1,310597183
5,109-2,310698367
6,109-3,356426204
7,109-year,1339254811
8,110-1,362410230
9,110-2,372145122


In [160]:
df_total_assets = get_acct_data_and_rename(df_balance, '資產總額', 'total_assets')
df_total_assets['avg_assets'] = (df_total_assets['total_assets'] + df_total_assets['total_assets'].shift(1)) / 2
df_total_assets['avg_assets'] = np.where(df_total_assets['avg_assets'].isnull(), df_total_assets['total_assets'], df_total_assets['avg_assets'])
df_total_assets

,year_season,total_assets,avg_assets
0,108-1,2187436785,2.187437e+09
1,108-2,2239343671,2.213390e+09
2,108-3,2134234450,2.186789e+09
3,108-4,2264805032,2.199520e+09
4,109-1,2343295696,2.304050e+09
5,109-2,2448664522,2.395980e+09
6,109-3,2635572214,2.542118e+09
7,109-4,2760711405,2.698142e+09
8,110-1,2919131461,2.839921e+09
9,110-2,3092815189,3.005973e+09


In [161]:
(5.005312e+09 + 5.508464e+09)/2

5256888000.0

In [162]:
df_total_assets_turnover = pd.merge(df_total_revenue, df_total_assets[['year_season', 'avg_assets']],  how='inner', on='year_season')
df_total_assets_turnover['assets_turnover_ratio'] = round(df_total_assets_turnover['total_revenue'] / df_total_assets_turnover['avg_assets'], 2)
df_total_assets_turnover

,year_season,total_revenue,avg_assets,assets_turnover_ratio
0,108-1,218704469,2.187437e+09,0.10
1,108-2,240998475,2.213390e+09,0.11
2,108-3,293045439,2.186789e+09,0.13
3,109-1,310597183,2.304050e+09,0.13
4,109-2,310698367,2.395980e+09,0.13
5,109-3,356426204,2.542118e+09,0.14
6,110-1,362410230,2.839921e+09,0.13
7,110-2,372145122,3.005973e+09,0.12
8,110-3,414670379,3.212564e+09,0.13
9,111-1,491075873,3.859090e+09,0.13


### Fixed Assets Turnover Ratio, FAT(固定資產周轉率)

- 固定資產周轉率 = 營業收入淨額÷固定資產平均淨值
- 固定資產平均淨值 = (期初淨值+期末淨值)÷2

In [163]:
# 固定資產: 不動產、廠房及設備
df_fixed_assets = get_acct_data_and_rename(df_balance, '不動產、廠房及設備', 'fixed_assets')
df_fixed_assets

,year_season,fixed_assets
0,108-1,1107651816
1,108-2,1142871184
2,108-3,1197955298
3,108-4,1352377405
4,109-1,1438215285
5,109-2,1493560950
6,109-3,1504438119
7,109-4,1555589120
8,110-1,1658551290
9,110-2,1722679133


In [164]:
df_total_revenue = get_acct_data_and_rename(df_profitloss, '營業收入合計', 'total_revenue')
df_total_revenue

,year_season,total_revenue
0,108-1,218704469
1,108-2,240998475
2,108-3,293045439
3,108-year,1069985448
4,109-1,310597183
5,109-2,310698367
6,109-3,356426204
7,109-year,1339254811
8,110-1,362410230
9,110-2,372145122


In [165]:
df_net_revenue = get_acct_data_and_rename(df_profitloss, '營業毛利（毛損）淨額', 'net_revenue')
df_net_revenue

,year_season,net_revenue
0,108-1,90357557
1,108-2,103730060
2,108-3,139412434
3,108-year,492701896
4,109-1,160776508
5,109-2,164623791
6,109-3,190493496
7,109-year,711130120
8,110-1,189839430
9,110-2,186196818


In [166]:
df_fixed_assets_turnover = pd.merge(df_total_revenue, df_fixed_assets,  how='inner', on='year_season')
df_fixed_assets_turnover['assets_turnover_ratio'] = round(df_fixed_assets_turnover['total_revenue'] / df_fixed_assets_turnover['fixed_assets'], 2)
df_fixed_assets_turnover

,year_season,total_revenue,fixed_assets,assets_turnover_ratio
0,108-1,218704469,1107651816,0.20
1,108-2,240998475,1142871184,0.21
2,108-3,293045439,1197955298,0.24
3,109-1,310597183,1438215285,0.22
4,109-2,310698367,1493560950,0.21
5,109-3,356426204,1504438119,0.24
6,110-1,362410230,1658551290,0.22
7,110-2,372145122,1722679133,0.22
8,110-3,414670379,1828894565,0.23
9,111-1,491075873,2104330654,0.23


# Profitability(獲利能力)

資產報酬率(Return of Assets, ROA) = [稅後淨利 + 利息費用 X ( 1 - 稅率 )] / 平均資產總額

根據玩股網的公式
分子僅算稅後淨利

稅後損益: post-tax profit, PAT
稅後淨利 = 稅前淨利 - 所得稅
利息費用: Interest Expense

In [167]:
# 稅前淨利（淨損）	:
df_pretax_profit = get_acct_data_and_rename(df_profitloss, '稅前淨利（淨損）', 'pre_tax_profit')

# 所得稅費用（利益）合計
df_income_tax = get_acct_data_and_rename(df_profitloss, '所得稅費用（利益）合計', 'income_tax')

# 稅後淨利: 本期淨利（淨損）
df_posttax_profit = pd.merge(df_pretax_profit, df_income_tax, how='inner', on='year_season')
df_posttax_profit['posttax_profit'] = df_posttax_profit['pre_tax_profit'] - df_posttax_profit['income_tax']
df_posttax_profit = pd.merge(df_posttax_profit, df_total_assets, how='inner', on='year_season')
df_posttax_profit['year'] = df_posttax_profit['year_season'].apply(lambda x : x.split('-')[0])
df_posttax_profit['season'] = df_posttax_profit['year_season'].apply(lambda x : x.split('-')[1])
df_posttax_profit

,year_season,pre_tax_profit,income_tax,posttax_profit,total_assets,avg_assets,year,season
0,108-1,68181652,6794342,61387310,2187436785,2.187437e+09,108,1
1,108-2,80545440,13769589,66775851,2239343671,2.213390e+09,108,2
2,108-3,112336271,11233817,101102454,2134234450,2.186789e+09,108,3
3,109-1,132147178,15084285,117062893,2343295696,2.304050e+09,109,1
4,109-2,136398638,15506124,120892514,2448664522,2.395980e+09,109,2
5,109-3,155124003,17745852,137378151,2635572214,2.542118e+09,109,3
6,110-1,155064389,15325640,139738749,2919131461,2.839921e+09,110,1
7,110-2,149391075,14909359,134481716,3092815189,3.005973e+09,110,2
8,110-3,173851639,17372485,156479154,3332311884,3.212564e+09,110,3
9,111-1,226831696,23958322,202873374,3992676667,3.859090e+09,111,1


In [168]:
df_112 = df_posttax_profit[df_posttax_profit['year_season'].str.startswith('112')]
avg_assets_total_112 = df_112['avg_assets'].mean()
posttax_profit_112 = df_112['posttax_profit'].sum()
df_112

,year_season,pre_tax_profit,income_tax,posttax_profit,total_assets,avg_assets,year,season
12,112-1,244274931,37325895,206949036,5045844348,5.005312e+09,112,1
13,112-2,214674909,32957903,181717006,5149465045,5.097655e+09,112,2
14,112-3,241940495,31145221,210795274,5484556381,5.317011e+09,112,3
20,112-4,278280989,39974788,238306201,5532371215,5.508464e+09,112,4


In [169]:
# 當季資產報酬率 = 當季poasttax_profit / (前一期total_assets + 當期total_assets)/2
238306201/((5532371215+5484556381)/2)

0.04326182575376526

In [175]:
df_each_year.sort_values('season', ascending=True)

,year_season,pre_tax_profit,income_tax,posttax_profit,total_assests,avg_assets,year,season
6,110-1,155064389,15325640,139738749,2919131461,2.839921e+09,110,1
7,110-2,149391075,14909359,134481716,3092815189,3.005973e+09,110,2
8,110-3,173851639,17372485,156479154,3332311884,3.212564e+09,110,3
18,110-4,184819211,18445696,166373515,3725503455,3.528908e+09,110,4


In [215]:
year_list = sorted({int(i[0]) for i in df_posttax_profit.year_season.str.split('-')})
year_list

[108, 109, 110, 111, 112, 113]

In [220]:
df_ROA = pd.DataFrame()
year_list = sorted({str(i[0]) for i in df_posttax_profit.year_season.str.split('-')})
for year in year_list[0:3]:
    df_each_year = df_posttax_profit[df_posttax_profit['year'] == year]
    season_list = df_posttax_profit[df_posttax_profit['year'] == year].season.to_list()
    if len(season_list) > 1:
        # calculate the return of assets(ROA) in each season in the year
        df_each_year = df_each_year.sort_values('season', ascending=True)
        for _, row in df_each_year.iterrows():
            row['ROA'] = row['posttax_profit'] / row['avg_assets']
            
        df_ROA = pd.concat([df_ROA, df_each_year], axis=0)

In [224]:
row

year_season              110-4
pre_tax_profit       184819211
income_tax            18445696
posttax_profit       166373515
total_assets        3725503455
avg_assets        3528907669.5
year                       110
season                       4
ROA                   0.047146
Name: 18, dtype: object

In [122]:
season_list = df_posttax_profit[df_posttax_profit['year'] == year].season.to_list()
len(season_list)

4

In [115]:
for year in year_list:
    print(year)

108
109
110
111
112
113


In [42]:
# dominator: the avg of previous season total assets and target season
def posttax_profit(df):
    # get the years in the data
    
    
    return 0